# Lesen, Verarbeiten und Schreiben von Dateien

Ein häufiger Anwendungsfall von Skriptsprachen ist die Verarbeitung von Daten, die vom Filesystem oder etwa per HTTP gelesen werden.

Im folgenden wollen wir alle gelungenen und misslungenen Login-Versuche ausgeben.

Zunächst öffnen wir die Datei und lesen die ersten fünf Zeilen:

In [24]:
file = open('auth.log', 'r')

for i in range(5):
    print(next(file))
   
# Datei schließen, um Ressourcen freizugeben
file.close()

Oct  6 00:17:01 jupiter CRON[79397]: pam_unix(cron:session): session opened for user root by (uid=0)

Oct  6 00:17:01 jupiter CRON[79397]: pam_unix(cron:session): session closed for user root

Oct  6 00:28:41 jupiter sshd[79416]: Received disconnect from 188.131.216.109 port 37954:11: Bye Bye [preauth]

Oct  6 00:28:41 jupiter sshd[79416]: Disconnected from authenticating user root 188.131.216.109 port 37954 [preauth]

Oct  6 01:02:58 jupiter sshd[79463]: Received disconnect from 133.130.107.88 port 52752:11: Normal Shutdown, Thank you for playing [preauth]



## Ressource-Leaks vermeiden: `with`
Der vorangehende Code hat den Nachteil, dass die Datei nicht geschlossen wird, wenn in der Schleife ein Fehler auftritt. 

Guter Stil ist es, die Anweisung `with` zu verwenden, um Ressourcen nach der Verwendung automatisch zu schließen:

In [ ]:
with open('auth.log', 'r') as file:
    for i in range(5):
        print(next(file))

## Filtern mit Regular Expressions
Nehmen wir an, wir interessieren uns für alle Logeinträge von `sshd` – das ist der SSH Dämon, der Anmeldeversuche am Server prüft – und die darin enthaltene IP-Adresse.

Dies ist ein klassischer Anwendungsfall für *Reguläre Ausdrücke*.

Die folgende Zelle sucht nach entsprechenden Einträgen in der Datei `auth.log` und bestimmt zusätzlich über die Datenbank `geolite2` das Herkunftsland des Versuchs: 

In [25]:
import operator
import re
from geoip import geolite2

byCountries = dict()
with open('auth.log', 'r') as file:
    for line in file:
        match = re.search('sshd.+ ([0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3})', line)
        if match:
            ip = match.group(1)
            location = geolite2.lookup(ip)
            if location:
                if location.country in byCountries:
                    byCountries[location.country] += 1
                else:
                    byCountries[location.country] = 1
                print("connection attempt from {0:15s} ({1})".format(ip, location.country))
            else:
                print("connection attempt from {0:15s} (unknown)".format(ip))

print()

# Sortiere die Länder nach der Zahl der Login-Versuche und gebe sie aus
sort = sorted(byCountries.items(), key=operator.itemgetter(1), reverse=True)
for country, attempts in sort:
    print("Land: {} => {:3d}".format(country, attempts))

connection attempt from 188.131.216.109 (RU)
connection attempt from 188.131.216.109 (RU)
connection attempt from 133.130.107.88  (JP)
connection attempt from 133.130.107.88  (JP)
connection attempt from 177.69.118.197  (BR)
connection attempt from 177.69.118.197  (BR)
connection attempt from 112.140.185.64  (SG)
connection attempt from 112.140.185.64  (SG)
connection attempt from 222.186.173.238 (CN)
connection attempt from 222.186.173.238 (CN)
connection attempt from 104.199.19.245  (US)
connection attempt from 60.12.215.85    (CN)
connection attempt from 60.12.215.85    (CN)
connection attempt from 51.255.174.215  (GB)
connection attempt from 51.255.174.215  (GB)
connection attempt from 222.186.173.238 (CN)
connection attempt from 222.186.173.238 (CN)
connection attempt from 202.51.110.214  (ID)
connection attempt from 202.51.110.214  (ID)
connection attempt from 212.176.114.10  (RU)
connection attempt from 212.176.114.10  (RU)
connection attempt from 173.220.206.162 (US)
connection

## Schreiben von Dateien
Das Ergebnis – die nach Ländern aufgeschlüsselte Anzahl von Login-Versuchen – wollen wir nun in eine Datei schreiben.

In [26]:
with open('result.txt', 'w') as out:
    for country, attempts in sort:
        out.write("Land: {} => {:3d}\n".format(country, attempts))

Manchmal wollen wir Daten auch in anderer Form, etwa als JSON, speichern:

In [ ]:
import json 

with open('result.json', 'w') as out:
    # Schreibe Dictionary als JSON in die Datei result.json
    json.dump(byCountries, out)